In [52]:
import geopandas as gpd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [53]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [54]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [55]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)

In [87]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-01-01', '2016-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2024 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2024-01-01', '2024-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])

In [57]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [58]:
def calc_evi(image):
    nir = image.select('B8')
    red = image.select('B4')
    blue = image.select('B2')
    evi = nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).multiply(2.5).rename('EVI')
    return image.addBands(evi)

In [88]:
change = satelite_2024.subtract(satelite_2016)

In [89]:
map_change = geemap.Map(center=[-11.5, -63.25], zoom=8)
map_change.addLayer(change)
map_change


Map(center=[-11.5, -63.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

No such comm: 32d56bd47b884a1ca7e6c4608c833fc2
No such comm: 9f7fc9e6d02041e6b7ce5ea2a6a462c4
No such comm: 9f7fc9e6d02041e6b7ce5ea2a6a462c4
No such comm: bf37095d370448f18825c19e30fd33e0
No such comm: bf37095d370448f18825c19e30fd33e0


Map(center=[-11.5, -63.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

Map(center=[-11.5, -63.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…